# Mechanics Tutorial 3: Coupling with Lumped Blood Circuits
![Pressure Volume Loop](3d0d-pv-loop.gif)

This tutorial shows how to couple 3d chamber models with 0d fluid models.

## Introduction

In this tutorial we will reproduce a simplified version of the model presented by [RegSalAfrFedDedQar:2022:cem](@citet).

> **Warning**
>
> The API for 3D-0D coupling is work in progress and is hence subject to potential breaking changes.

## Commented Program
We start by loading Thunderbolt and LinearSolve to use a custom direct solver of our choice.

In [1]:
using Thunderbolt, LinearSolve

Finally, we try to approach a valid initial state by solving a simpler model first.

In [2]:
using OrdinaryDiffEqTsit5, OrdinaryDiffEqOperatorSplitting

fluid_model_init = RSAFDQ2022LumpedCicuitModel()
u0 = zeros(Thunderbolt.num_states(fluid_model_init))
Thunderbolt.default_initial_condition!(u0, fluid_model_init)
prob = ODEProblem((du, u, p, t) -> Thunderbolt.lumped_driver!(du, u, t, [], p), u0, (0.0, 100*fluid_model_init.THB), fluid_model_init)
sol = solve(prob, Tsit5())

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 11931-element Vector{Float64}:
     0.0
     0.04838838098003728
     0.2792640643984848
     0.7267523496016612
     1.3438604381655068
     2.144258889210636
     3.155553117293995
     4.398246405764439
     5.905268886754168
     7.716402419487973
     ⋮
 79949.53923019694
 79956.29777079161
 79963.05665516232
 79969.81596119265
 79976.57556892763
 79983.3352996462
 79990.09501501129
 79996.8546516533
 80000.0
u: 11931-element Vector{Vector{Float64}}:
 [65.0, 120.0, 65.0, 145.0, 10.66, 4.0, 4.67, 3.2, 0.0, 0.0, 0.0, 0.0]
 [64.99775599207983, 120.00227770768569, 64.99411468494874, 145.00593718603966, 10.659998698778578, 3.9999999665140544, 4.669999657362968, 3.199999933568232, 0.0004815730430550299, 0.002134255662419863, 0.001058704949551402, 0.0013873724257603324]
 [64.98801864146336, 120.01307519006716, 64.96751009486455, 145.03415042166324, 10.65995732473332, 3.9999989362827226, 4.66998887341767, 3.1999

plot(sol, idxs=[1,2,3,4], tspan=(99*fluid_model_init.THB, 100*fluid_model_init.THB))

In [3]:
# Precomputed initial guess
u₀fluid = sol.u[end]
@info "Total blood volume: $(sum(u₀fluid[1:4])) + $(fluid_model_init.Csysₐᵣ*u₀fluid[5]) + $(fluid_model_init.Csysᵥₑₙ*u₀fluid[6]) + $(fluid_model_init.Cpulₐᵣ*u₀fluid[7]) + $(fluid_model_init.Cpulᵥₑₙ*u₀fluid[8])"

[ Info: Total blood volume: 393.1889560892115 + 101.66345564540934 + 4788.81739818597 + 362.1295874619882 + 379.8536497430151


We now generate the mechanical subproblem as in the first tutorial

In [4]:
scaling_factor = 3.7;

> **Warning**
>
> Tuning parameter until all bugs are fixed in this tutorial :)

In [5]:
mesh = generate_ideal_lv_mesh(8,2,5;
    inner_radius = scaling_factor*0.7,
    outer_radius = scaling_factor*1.0,
    longitudinal_upper = 0.4,
    apex_inner = scaling_factor* 1.3,
    apex_outer = scaling_factor*1.5
)
mesh = Thunderbolt.hexahedralize(mesh)

Thunderbolt.SimpleMesh{3, Hexahedron, Float64} with 736 Hexahedron cells and 965 nodes
  Surface subdomains:
    Epicardium 184 Hexahedron cells
    Base 64 Hexahedron cells
    Endocardium 184 Hexahedron cells


> **Todo**
>
> The 3D0D coupling does not yet support multiple subdomains.

In [6]:
coordinate_system = compute_lv_coordinate_system(mesh)
microstructure    = create_microstructure_model(
    coordinate_system,
    LagrangeCollection{1}()^3,
    ODB25LTMicrostructureParameters(),
);
passive_material_model = Guccione1991PassiveModel()
active_material_model  = Guccione1993ActiveModel()
function calcium_profile_function(x::LVCoordinate,t_global)
    linear_interpolation(t,y1,y2,t1,t2) = y1 + (t-t1) * (y2-y1)/(t2-t1)
    ca_peak(x)                          = 1.0
    t = t_global % 800.0
    if 0 ≤ t ≤ 120.0
        return linear_interpolation(t,        0.0, ca_peak(x),   0.0, 120.0)
    elseif t ≤ 272.0
        return linear_interpolation(t, ca_peak(x),        0.0, 120.0, 272.0)
    else
        return 0.0
    end
end
calcium_field = AnalyticalCoefficient(
    calcium_profile_function,
    coordinate_system,
)
sarcomere_model = CaDrivenInternalSarcomereModel(ConstantStretchModel(), calcium_field)
active_stress_model = ActiveStressModel(
    passive_material_model,
    active_material_model,
    sarcomere_model,
    microstructure,
)
weak_boundary_conditions = (RobinBC(1.0, "Epicardium"),NormalSpringBC(100.0, "Base"))
solid_model = QuasiStaticModel(:displacement, active_stress_model, weak_boundary_conditions);

The solid model is now couple with the circuit model by adding a Lagrange multipliers constraining the 3D chamber volume to match the chamber volume in the 0D model.

In [7]:
fluid_model = RSAFDQ2022LumpedCicuitModel(; lv_pressure_given = false)
coupler = LumpedFluidSolidCoupler(
    [
        ChamberVolumeCoupling(
            "Endocardium",
            RSAFDQ2022SurrogateVolume(),
            :Vₗᵥ,
            :pₗᵥ,
        )
    ],
    :displacement,
)
coupled_model = RSAFDQ2022Model(solid_model, fluid_model, coupler);

> **Todo**
>
> Once we figure out a nicer way to do this we should add more detailed docs here.

Now we semidiscretize the model spatially as usual with finite elements and annotate the model with a stable split.

In [8]:
spatial_discretization_method = FiniteElementDiscretization(
    Dict(:displacement => LagrangeCollection{1}()^3),
    [
        Dirichlet(:displacement, getfacetset(mesh, "Base"), (x,t) -> [0.0], [3]),
        Dirichlet(:displacement, getnodeset(mesh, "MyocardialAnchor1"), (x,t) -> (0.0, 0.0, 0.0), [1,2,3]),
        Dirichlet(:displacement, getnodeset(mesh, "MyocardialAnchor2"), (x,t) -> (0.0, 0.0), [2,3]),
        Dirichlet(:displacement, getnodeset(mesh, "MyocardialAnchor3"), (x,t) -> (0.0,), [3]),
        Dirichlet(:displacement, getnodeset(mesh, "MyocardialAnchor4"), (x,t) -> (0.0,), [3])
    ],
)
splitform = semidiscretize(
    RSAFDQ2022Split(coupled_model),
    spatial_discretization_method,
    mesh,
)

dt₀ = 1.0
dtvis = 10.0
tspan = (0.0, 3*800.0)

(0.0, 2400.0)

This speeds up the CI # hide

In [9]:
tspan = (0.0, 10.0)    # hide

(0.0, 10.0)

The remaining code is very similar to how we use SciML solvers.

In [10]:
chamber_solver = HomotopyPathSolver(
    NewtonRaphsonSolver(;
        max_iter=10,
        tol=1e-2,
        inner_solver=SchurComplementLinearSolver(
            LinearSolve.UMFPACKFactorization()
        )
    )
)
blood_circuit_solver = Tsit5()
timestepper = LieTrotterGodunov((chamber_solver, blood_circuit_solver))

u₀ = zeros(solution_size(splitform))
u₀solid_view = @view  u₀[OS.get_solution_indices(splitform, 1)]
u₀fluid_view = @view  u₀[OS.get_solution_indices(splitform, 2)]
u₀fluid_view .= u₀fluid

problem = OperatorSplittingProblem(splitform, u₀, tspan)
integrator = init(problem, timestepper, dt=dt₀, verbose=true; dtmax=10.0);

# f2 = Figure()
# axs = [
#     Axis(f2[1, 1], title="LV"),
#     Axis(f2[1, 2], title="RV"),
#     Axis(f2[2, 1], title="LA"),
#     Axis(f2[2, 2], title="RA")
# ]

# vlv = Observable(Float64[])
# plv = Observable(Float64[])

# vrv = Observable(Float64[])
# prv = Observable(Float64[])

# vla = Observable(Float64[])
# pla = Observable(Float64[])

# vra = Observable(Float64[])
# pra = Observable(Float64[])

# lines!(axs[1], vlv, plv)
# lines!(axs[2], vrv, prv)
# lines!(axs[3], vla, pla)
# lines!(axs[4], vra, pra)
# for i in 1:4
#     xlims!(axs[1], 0.0, 180.0)
#     ylims!(axs[1], 0.0, 180.0)
# end
# display(f2)

[ Info: Volume 3D 112.23695369430204
[ Info: Chamber 1 p=0.0, V0=112.23695369430216
[ Info: Volume 3D 112.23695369430214
[ Info: Chamber 1 p=2.0702088767869447e-15, V0=112.23695369430216


> **Todo**
>
> recover online visualization of the pressure volume loop

> **Todo**
>
> The post-processing API is not yet finished.
> Please revisit the tutorial later to see how to post-process the simulation online.
> Right now the solution is just exported into VTK, such that users can visualize the solution in e.g. ParaView.

Now we can finally solve the coupled problem in time.

In [11]:
io = ParaViewWriter("CM03_3d0d-coupling");
for (u, t) in TimeChoiceIterator(integrator, tspan[1]:dtvis:tspan[2])
    chamber_function = OS.get_operator(splitform, 1)
    (; dh) = chamber_function.structural_function
    store_timestep!(io, t, dh.grid)
    usolid_view = @view u[OS.get_solution_indices(splitform, 1)]
    Thunderbolt.store_timestep_field!(io, t, dh, usolid_view, :displacement)
    Thunderbolt.finalize_timestep!(io, t)

    # if t > 0.0
    #     lv = chamber_function.tying_info.chambers[1]
    #     append!(vlv.val, lv.V⁰ᴰval)
    #     append!(plv.val, u[lv.pressure_dof_index_global])
    #     notify(vlv)
    #     notify(plv)
    # end
    # TODO plot other chambers
end

[ Info: Volume 3D 112.23695369430204
[ Info: Chamber 1 p=0.0, V0=117.55380932929918
[ Info: Volume 3D 117.62016991021405
[ Info: Chamber 1 p=0.3233629323229525, V0=117.55380932929918
[ Info: Volume 3D 117.55381416932974
[ Info: Chamber 1 p=0.3177275539569428, V0=117.55380932929918
[ Info: Volume 3D 117.55380907641515
[ Info: Chamber 1 p=0.3177469760810646, V0=117.55380932929918
[ Info: Volume 3D 117.55380907641515
[ Info: Chamber 1 p=0.3177469760810646, V0=118.4818982023052
[ Info: Volume 3D 118.48735051236947
[ Info: Chamber 1 p=0.5585722933019472, V0=118.4818982023052
[ Info: Volume 3D 118.48191739090099
[ Info: Chamber 1 p=0.5582721164084168, V0=118.4818982023052
[ Info: Volume 3D 118.48191739090099
[ Info: Chamber 1 p=0.5582721164084168, V0=119.15406122905411
[ Info: Volume 3D 119.15813146477885
[ Info: Chamber 1 p=0.7932402095960721, V0=119.15406122905411
[ Info: Volume 3D 119.15407803499465
[ Info: Chamber 1 p=0.7930988860780426, V0=119.15406122905411
[ Info: Volume 3D 119.154078

> **Tip**
>
> If you want to see more details of the solution process launch Julia with Thunderbolt as debug module:
> ```
> JULIA_DEBUG=Thunderbolt julia --project --threads=auto my_simulation_runner.jl
> ```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*